In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
# Configuración del directorio y parámetros de las imágenes
data_dir = './dataset'  # Ruta al dataset que tienes en tu PC
image_size = (224, 224)  # Tamaño de las imágenes que utilizará el modelo
batch_size = 32  # Tamaño del lote para entrenamiento
num_classes = 8  # Número de clases

In [3]:
# Generador de datos con separación de entrenamiento y validación
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalización y división

# Conjunto de entrenamiento
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Multi-clase para reconocer tipo de fruta y frescura
    subset='training'
)

# Conjunto de validación
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 9695 images belonging to 8 classes.
Found 2420 images belonging to 8 classes.


In [4]:
# Crear lista de nombres de clases basados en el orden de los índices
class_names = list(train_generator.class_indices.keys())
print(class_names)


['fresh_apple', 'fresh_banana', 'fresh_orange', 'fresh_tomato', 'stale_apple', 'stale_banana', 'stale_orange', 'stale_tomato']


In [5]:
# Definición del modelo de red neuronal convolucional (CNN)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # 'num_classes' es el número total de categorías (fresco/pasado + tipo de fruta)
])

C:\Users\Erick\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Compilación del modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Entrenamiento del modelo
epochs = 30
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

C:\Users\Erick\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 316s 1s/step - accuracy: 0.5185 - loss: 1.3616 - val_accuracy: 0.7240 - val_loss: 0.6434
Epoch 2/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 322s 1s/step - accuracy: 0.8033 - loss: 0.5527 - val_accuracy: 0.8496 - val_loss: 0.3286
Epoch 3/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 318s 1s/step - accuracy: 0.8660 - loss: 0.3695 - val_accuracy: 0.8033 - val_loss: 0.3842
Epoch 4/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 293s 966ms/step - accuracy: 0.8783 - loss: 0.3162 - val_accuracy: 0.8731 - val_loss: 0.2687
Epoch 5/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 295s 975ms/step - accuracy: 0.8991 - loss: 0.2627 - val_accuracy: 0.8368 - val_loss: 0.4792
Epoch 6/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 333s 1s/step - accuracy: 0.9123 - loss: 0.2279 - val_accuracy: 0.8988 - val_loss: 0.2726
Epoch 7/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 320s 1s/step - accuracy: 0.9160 - loss: 0.2231 - val_accuracy: 0.9322 - val_loss: 0.1436
Epoch 8/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 323s 1s/step - accuracy: 0.9170 - loss: 0.2180 - va

In [8]:
# Evaluación del modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Loss en validación: {val_loss}")
print(f"Precisión en validación: {val_accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 15s 199ms/step - accuracy: 0.9821 - loss: 0.0601
Loss en validación: 0.05402000993490219
Precisión en validación: 0.9847107529640198


In [9]:
# Guardar el modelo entrenado
model.save("modelo_clasificacion_frutas.h5")